In [138]:
import sys
sys.path.append("../")

import pandas as pd
import seaborn as sns
import numpy as np

sns.set_style('darkgrid')
sns.set(font_scale = 1.5)

In [139]:
from src.finman.utils.gsheet import GSheetWorker

In [140]:
sheet_id = "1GqwNPuOtQUXfltK6F4oHpx-U1BqBgiDZfC5YfDcJyto"
gcreds_file = "../secrets/finman-433017-da308d823497.json"

gsw = GSheetWorker(gcreds_file)

# 1. Data Processing

### 1.1 Чтение всех транзакций из гуглодока

In [141]:
sheet_setups = [
    {
        "bank_sheet_name": "t-bank (main)",
        "dt_col": "Время операции",
        "trans_rub_col": "Сумма операции в валюте карты",
    },
    {
        "bank_sheet_name": "t-bank (deposit)",
        "dt_col": "Время операции",
        "trans_rub_col": "Сумма операции в валюте карты",
    },
    {
        "bank_sheet_name": "t-bank (invest)",
        "dt_col": "Время операции",
        "trans_rub_col": "Сумма операции в валюте карты",
    },
    {
        "bank_sheet_name": "ozon-bank (main)",
        "dt_col": "Время операции",
        "trans_rub_col": "Сумма операции",
    },
]

transaction_lists = [
    "t-bank (main)", "ozon-bank (main)"
]

cat_col = "Категория"
subcat_col = "Подкатегория"
debt_col = "Долг"

In [142]:
trans_df = []
for setup in sheet_setups:
    gs_df = gsw.get_df(sheet_id, setup["bank_sheet_name"])

    gs_df = gs_df.rename(columns={
        setup['dt_col']: "dt",
        setup['trans_rub_col']: "trans_rub",

    })

    if setup["bank_sheet_name"] in transaction_lists:
        trans_df.append(gs_df[[
            "dt", "trans_rub", cat_col, subcat_col, debt_col
        ]])
    else:
        trans_df.append(gs_df[[
            "dt", "trans_rub", cat_col, subcat_col
        ]])

    trans_df[-1]['account'] = setup["bank_sheet_name"]

    print(setup["bank_sheet_name"] + " done!")

trans_df = pd.concat(trans_df)

/var/folders/n9/7qzl9sfj3_n9hmnht95_9jy80000gn/T/ipykernel_3547/27192228.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



t-bank (main) done!


/var/folders/n9/7qzl9sfj3_n9hmnht95_9jy80000gn/T/ipykernel_3547/27192228.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



t-bank (deposit) done!


/var/folders/n9/7qzl9sfj3_n9hmnht95_9jy80000gn/T/ipykernel_3547/27192228.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



t-bank (invest) done!
ozon-bank (main) done!


/var/folders/n9/7qzl9sfj3_n9hmnht95_9jy80000gn/T/ipykernel_3547/27192228.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [143]:
trans_df.head()

,dt,trans_rub,Категория,Подкатегория,Долг,account
0,2024-10-30 20:27,-4000,Transfer,Internal,None,t-bank (main)
1,2024-10-30 12:57,-2283.1,Food & Dining,Delivery,None,t-bank (main)
2,2024-10-30 11:39,-2730,Entertainment,Restaurants,None,t-bank (main)
3,2024-10-29 21:53,-900,Entertainment,Restaurants,None,t-bank (main)
4,2024-10-29 20:49,-50,Sport,Padel,None,t-bank (main)


In [144]:
trans_df['dt'] = pd.to_datetime(trans_df['dt'])
trans_df['trans_rub'] = pd.to_numeric(trans_df['trans_rub'])

### 1.2 Агрегация по долгам

In [145]:
trans_df_no_debt = trans_df.loc[(trans_df['Долг'].isnull()) | (trans_df['Долг'] == '')]
trans_df_debt = trans_df.loc[(trans_df['Долг'] != '') & (trans_df['Долг'].notnull())]

assert (trans_df_no_debt.shape[0] + trans_df_debt.shape[0]) == trans_df.shape[0]

In [146]:
trans_df_debt = trans_df_debt.groupby('Долг').agg({
    'dt': min,
    'trans_rub': sum,
    'Категория': "last",
    'Подкатегория': "last",
    'account': lambda x: ";".join(set(x))
}).reset_index()

In [147]:
trans_df_debt

,Долг,dt,trans_rub,Категория,Подкатегория,account
0,Padel_240903,2024-08-28 10:55:00,-1500.0,Sport,Padel,t-bank (main)
1,egg_s 240925,2024-09-25 11:03:00,-1540.0,Entertainment,Restaurants,t-bank (main)
2,костюм в яки,2024-10-15 17:58:00,-12500.0,Other,Other,t-bank (main)
3,кэш_алене 240913,2024-09-13 20:06:00,0.0,Transfer,Debt,t-bank (main)
4,мак к зубрам,2024-10-19 13:54:00,-2554.0,Entertainment,Restaurants,t-bank (main)
5,падел фили 240807,2024-08-05 17:02:00,-2975.0,Sport,Padel,t-bank (main)
6,падел240816,2024-08-13 11:29:00,-6000.0,Sport,Padel,t-bank (main)
7,падел_241004,2024-10-04 15:46:00,-800.0,Sport,Padel,t-bank (main)
8,падел_241022,2024-10-22 11:14:00,-3400.0,Sport,Padel,t-bank (main)
9,подарок на годовщину2024,2024-07-21 16:21:00,15000.0,Debt,Other,t-bank (main)


In [148]:
trans_df_debt_free = pd.concat((trans_df_debt, trans_df_no_debt))

### 1.3 Фильтруем только транзакции для отчета

In [149]:
exp_mask = (trans_df_debt_free['trans_rub'] < 0) & (trans_df_debt_free['account'].isin(transaction_lists) & \
                                                    (trans_df_debt_free['Категория'] != "Transfer") & \
                                                    (trans_df_debt_free['Подкатегория'] != "Internal"))
inc_mask = (trans_df_debt_free['trans_rub'] > 0)

exp_df = trans_df_debt_free.loc[exp_mask]
exp_df['trans_rub'] = exp_df['trans_rub'].abs()

/var/folders/n9/7qzl9sfj3_n9hmnht95_9jy80000gn/T/ipykernel_3547/2815616887.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [150]:
months_dict = {key: value for key,value in zip(np.arange(1, 13), ["январь", "февраль", "март", "апрель", "май", "июнь", 
                                                                    "июль", "август", "сентябрь", "октябрь", "ноябрь", "декабрь"])}

In [151]:
months_dict

{1: 'январь',
 2: 'февраль',
 3: 'март',
 4: 'апрель',
 5: 'май',
 6: 'июнь',
 7: 'июль',
 8: 'август',
 9: 'сентябрь',
 10: 'октябрь',
 11: 'ноябрь',
 12: 'декабрь'}

# Пончик трат по категориям

In [152]:
month = 9

In [153]:
df_vis = exp_df.loc[exp_df['dt'].dt.month == month]

In [35]:
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import Output

# Sample data creation (you can replace this with your actual DataFrame)
data = {
    "date": ["2023-01-01 10:00:00", "2023-01-01 11:00:00", "2023-01-02 12:00:00", "2023-01-03 13:00:00"],
    "money_spent": [100, 150, 200, 50],
    "category": ["Food", "Entertainment", "Food", "Utilities"],
    "subcategory": ["Groceries", "Movies", "Restaurants", "Electricity"]
}
df = pd.DataFrame(data)

# Output widget for displaying interactive figures
output = Output()

# Function to generate category pie chart
def create_category_pie():
    category_data = df.groupby('category')['money_spent'].sum().reset_index()
    
    fig = go.FigureWidget(go.Pie(
        labels=category_data['category'],
        values=category_data['money_spent'],
        hole=0.3
    ))

    fig.update_layout(title="Transaction Distribution by Category")
    
    # Click event for category pie
    fig.data[0].on_click(on_category_click)
    with output:
        output.clear_output(wait=True)
        display(fig)

# Function to generate subcategory pie chart based on selected category
def create_subcategory_pie(selected_category):
    subcategory_data = df[df['category'] == selected_category].groupby('subcategory')['money_spent'].sum().reset_index()

    fig = go.FigureWidget(go.Pie(
        labels=subcategory_data['subcategory'],
        values=subcategory_data['money_spent'],
        hole=0.3
    ))

    fig.update_layout(title=f"Transaction Distribution in {selected_category} by Subcategory")
    
    # Click event to go back to main category pie
    fig.data[0].on_click(on_subcategory_click)
    with output:
        output.clear_output(wait=True)
        display(fig)

# Event handler when a category slice is clicked
def on_category_click(trace, points, state):
    selected_category = trace.labels[points.point_inds[0]]
    create_subcategory_pie(selected_category)

# Event handler to go back to category view
def on_subcategory_click(trace, points, state):
    create_category_pie()

# Display the initial category pie chart
create_category_pie()
output


Output()

In [34]:
import plotly.graph_objects as go
import pandas as pd

# Sample data frame
data = {
    'date_time': ['2024-01-01 10:00:00', '2024-01-01 11:00:00', '2024-01-02 12:00:00',
                  '2024-01-02 13:00:00', '2024-01-03 14:00:00', '2024-01-03 15:00:00'],
    'amount': [100, 150, 200, 130, 120, 110],
    'category': ['Food', 'Transport', 'Food', 'Entertainment', 'Transport', 'Food'],
    'subcategory': ['Groceries', 'Bus', 'Dining Out', 'Movies', 'Train', 'Dining Out']
}

df = pd.DataFrame(data)

# Function to plot the main category pie chart
def plot_category_pie(df):
    # Grouping data by category
    category_data = df.groupby('category').sum()['amount']
    
    # Creating the pie chart
    fig = go.Figure(data=[go.Pie(labels=category_data.index, values=category_data.values, 
                                 hole=0.3)])

    fig.update_traces(hoverinfo='label+percent', textinfo='label+percent', 
                      textfont_size=15)

    # Adding callback function for click event
    fig.update_layout(title_text="Transaction Distribution by Category")
    fig.data[0].on_click(lambda trace, points, state: plot_subcategory_pie(df, points))

    return fig

# Function to plot the subcategory pie chart
def plot_subcategory_pie(df, points):
    category_selected = points.point_inds[0]
    category_name = df['category'].unique()[category_selected]
    
    # Filter data based on selected category
    subcategory_data = df[df['category'] == category_name].groupby('subcategory').sum()['amount']

    # Creating the pie chart
    fig = go.Figure(data=[go.Pie(labels=subcategory_data.index, values=subcategory_data.values, 
                                 hole=0.3)])
    
    fig.update_traces(hoverinfo='label+percent', textinfo='label+percent', 
                      textfont_size=15,
                      pull=[0.1 if subcategory_data.index[i] == 'Dining Out' else 0 for i in range(len(subcategory_data))])
    
    # Adding callback function for click event
    fig.update_layout(title_text=f"Transaction Distribution for {category_name} by Subcategory")
    fig.data[0].on_click(lambda trace, points, state: plot_category_pie(df))
    
    fig.show()

# Plotting the initial pie chart
fig = plot_category_pie(df)
fig.show()


In [37]:
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import Output

# Sample data (you can replace this with your actual DataFrame)
data = {
    "date": ["2023-01-01 10:00:00", "2023-01-01 11:00:00", "2023-01-02 12:00:00", "2023-01-03 13:00:00"],
    "money_spent": [100, 150, 200, 50],
    "category": ["Food", "Entertainment", "Food", "Utilities"],
    "subcategory": ["Groceries", "Movies", "Restaurants", "Electricity"]
}
df = pd.DataFrame(data)

# Output widget for displaying interactive figures
output = Output()

# Function to generate category waterfall chart
def create_category_waterfall():
    category_data = df.groupby('category')['money_spent'].sum().reset_index()
    
    fig = go.FigureWidget(go.Waterfall(
        x=category_data['category'],
        y=category_data['money_spent'],
        connector=dict(line=dict(color="black", dash="dot"))
    ))

    fig.update_layout(title="Transaction Distribution by Category (Waterfall Chart)")
    
    # Click event for category waterfall
    fig.data[0].on_click(on_category_click)
    with output:
        output.clear_output(wait=True)
        display(fig)

# Function to generate subcategory waterfall chart based on selected category
def create_subcategory_waterfall(selected_category):
    subcategory_data = df[df['category'] == selected_category].groupby('subcategory')['money_spent'].sum().reset_index()

    fig = go.FigureWidget(go.Waterfall(
        x=subcategory_data['subcategory'],
        y=subcategory_data['money_spent'],
        connector=dict(line=dict(color="black", dash="dot"))
    ))

    fig.update_layout(title=f"Transaction Distribution in {selected_category} by Subcategory (Waterfall Chart)")
    
    # Click event to go back to main category waterfall
    fig.data[0].on_click(on_subcategory_click)
    with output:
        output.clear_output(wait=True)
        display(fig)

# Event handler when a category bar is clicked
def on_category_click(trace, points, state):
    selected_category = trace.x[points.point_inds[0]]
    create_subcategory_waterfall(selected_category)

# Event handler to go back to category view
def on_subcategory_click(trace, points, state):
    create_category_waterfall()

# Display the initial category waterfall chart
create_category_waterfall()
output


Output()

In [38]:
df_vis

,dt,trans_rub,Категория,Подкатегория,Долг,account
168,2024-09-30 23:50:00,362.01,Transportation,Carsharing,None,t-bank (main)
169,2024-09-30 20:01:00,2983.00,Food & Dining,Delivery,None,t-bank (main)
170,2024-09-30 19:00:00,989.00,Other,None,None,t-bank (main)
172,2024-09-29 20:40:00,123.21,Transportation,Carsharing,None,t-bank (main)
173,2024-09-29 19:01:00,2261.00,Entertainment,Restaurants,None,t-bank (main)
...,...,...,...,...,...,...
335,2024-09-01 19:32:00,7480.00,Entertainment,Restaurants,None,t-bank (main)
336,2024-09-01 17:57:00,730.59,Transportation,Carsharing,None,t-bank (main)
337,2024-09-01 17:21:00,3564.00,Shopping,Gifts,None,t-bank (main)
338,2024-09-01 12:54:00,100.00,Personal,Subscriptions,None,t-bank (main)


In [18]:
import plotly.express as px

# fig = px.pie(trans_df.loc[exp_mask & (trans_df['dt'].dt.month == month)], values='trans_rub', names='Категория', title=f'Траты за месяц {month}')
# df_tmp = trans_df.loc[exp_mask]

fig = px.pie(df_vis, values='trans_rub', names='Категория', title=f'Всего трат за {months_dict[month]}: {df_vis["trans_rub"].sum()}')
fig.show()

In [19]:
df_vis_2 = df_vis.loc[df_vis['Подкатегория'] != "Mortgage/Rent"]

fig = px.pie(df_vis_2, values='trans_rub', names='Категория', title=f'Всего трат за {months_dict[month]}: {df_vis_2["trans_rub"].sum()}')
fig.show()

,dt,trans_rub,Категория,Подкатегория,Долг,account
0,2024-08-19 12:45:00,-725.85,Food & Dining,Business Lunch,None,t-bank (main)
1,2024-08-19 09:06:00,-1103.00,Food & Dining,Delivery,None,t-bank (main)
2,2024-08-18 19:46:00,-197.00,Transportation,Carsharing,None,t-bank (main)
3,2024-08-18 13:16:00,-640.00,Household,Groceries,,t-bank (main)
4,2024-08-18 13:12:00,-580.00,Household,Groceries,None,t-bank (main)
...,...,...,...,...,...,...
42,2024-04-16 16:56:00,-1495.00,Transfer,None,None,ozon-bank (main)
43,2024-04-15 22:38:00,-1302.00,Household,Accessories,None,ozon-bank (main)
44,2024-04-15 08:45:00,-2363.00,Household,Accessories,None,ozon-bank (main)
45,2024-04-13 15:32:00,-4840.00,Household,Accessories,None,ozon-bank (main)


In [154]:
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import Output


# Output widget for displaying interactive figures
output = Output()

# Function to generate category waterfall chart
def create_category_waterfall():
    category_data = df_vis.groupby('Категория')['trans_rub'].sum().reset_index()
    
    fig = go.FigureWidget(go.Waterfall(
        x=category_data['Категория'],
        y=category_data['trans_rub'],
        connector=dict(line=dict(color="black", dash="dot"))
    ))

    fig.update_layout(title="Transaction Distribution by Category (Waterfall Chart)")
    
    # Click event for category waterfall
    fig.data[0].on_click(on_category_click)
    with output:
        output.clear_output(wait=True)
        display(fig)

# Function to generate subcategory waterfall chart based on selected category
def create_subcategory_waterfall(selected_category):
    subcategory_data = df_vis[df_vis['Категория'] == selected_category].groupby('Подкатегория')['trans_rub'].sum().reset_index()

    fig = go.FigureWidget(go.Waterfall(
        x=subcategory_data['Подкатегория'],
        y=subcategory_data['trans_rub'],
        connector=dict(line=dict(color="black", dash="dot"))
    ))

    fig.update_layout(title=f"Transaction Distribution in {selected_category} by Subcategory (Waterfall Chart)")
    
    # Click event to go back to main category waterfall
    fig.data[0].on_click(on_subcategory_click)
    with output:
        output.clear_output(wait=True)
        display(fig)

# Event handler when a category bar is clicked
def on_category_click(trace, points, state):
    selected_category = trace.x[points.point_inds[0]]
    create_subcategory_waterfall(selected_category)

# Event handler to go back to category view
def on_subcategory_click(trace, points, state):
    create_category_waterfall()

# Display the initial category waterfall chart
create_category_waterfall()
output


Output()